In [252]:
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup

In [253]:
session = requests.session()
ua = UserAgent(verify_ssl=False)

In [254]:
def parser(url, sentiment):
    headers = {'User-Agent': ua.random}
    response = session.get(url, headers=headers)
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    reviews = soup.find_all('span', {'itemprop': 'reviewBody'})
    for review in reviews:
        rev.append(review.text)
        sent.append(sentiment)

##### Я буду скачивать данные с 6 разных страничек, по 15 на каждой, поэтому, сложил их разную часть ссылки в словарик с типом отзывов на данной странице

In [255]:
# список для отзывов
rev = []
# список для тональностей отзывов
sent = []
dict_urls = {
    "?page=3&sort=negative": 'neg',
    "?page=4&sort=negative": 'neg',
    "?page=5&sort=negative": 'neg',
    "?page=3&sort=positive": 'pos',
    "?page=4&sort=positive": 'pos',
    "?page=5&sort=positive": 'pos'
}

In [256]:
for key, value in dict_urls.items():
    #соединяем инвариантную и вариативную часть ссылки
    full_url = 'https://vseotzyvy.ru/item/547/reviews-igra-world-of-tanks/' + key
    #закидываем в парсер ссылку и тип отзывов на ней
    parser(full_url, value)

#### Токенизация, нижний регистр, лемматизация

In [257]:
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

for i in range(len(rev)):
    words = [w.lower() for w in word_tokenize(rev[i]) if w.isalpha()]
    list_of_lemmas = []
    for word in words:
        ana = morph.parse(word)
        first = ana[0]
        lemma = first.normal_form
        list_of_lemmas.append(lemma)
    rev[i] = list_of_lemmas

#### Разбиваю на трейн и тест, как при обучении модели (чтобы потом посчитать accuracy)

In [300]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(rev, sent, test_size=0.111)

In [333]:
# частотные словари
dict_pos = {}
dict_neg = {}
# множества для слов из только негативных и только положительных отзывов
final_set_pos = set()
final_set_neg = set()

#### Создаю частотные словари для лемм в положительных и негативных отзывах

In [334]:
for i in range(len(X_train)):
    if y_train[i] == 'pos':
        for token in X_train[i]:
            if token not in dict_pos.keys():
                dict_pos[token] = 1
            else:
                dict_pos[token] += 1
    else:
        for token in X_train[i]:
            if token not in dict_neg.keys():
                dict_neg[token] = 1
            else:
                dict_neg[token] += 1

#### Если проверять, больше ли 2 слов в частотном словаре, то точность обычно ниже, а в множествах оказывается довольно мало слов, поэтому выбрасывается только самый мусор, а от 2 уже пробуем учитывать

In [335]:
for key in dict_pos.keys():
    if dict_pos[key] > 1 and key not in dict_neg.keys():
        final_set_pos.add(key)

for key in dict_neg.keys():
    if dict_neg[key] > 1 and key not in dict_pos.keys():
        final_set_neg.add(key)

In [336]:
y_pred = []

#### Определяем тональность отзывов из трейна

In [337]:
for text in X_test:
    # Создаем счетчик: каждое "положительное" слово +1, "негативное" -1
    counter = 0
    for token in text:
        if token in final_set_pos:
            counter += 1
        elif token in final_set_neg:
            counter -= 1
    #Теперь по итоговой сумме видно, было больше "положительных" или "негативных" слов
    if counter > 0:
        y_pred.append('pos')
    elif counter < 0:
        y_pred.append('neg')
    #При ничьей, наверное, не стоит решать наобум, хотя так может быть резлуьтативней, введем нейтральую тональность
    else:
        y_pred.append('neut')

In [338]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)
#вообще точность варьируется от 0,5 до 1 без улучшений (в зависимости от тестовой выборки)
#кажется, 7 выглядит как +- среднее

0.7

### Хочу написать небольшой комментарий. Я немного смотрел отзывы, и было некоторое количество, где отзыв явно негативный, а оценка, видимо по ошибке, стоит положительная. Чисто человеческий фактор, получается. По идее, это должно создавать некоторые проблемы, например, во множество негативных слов попадет меньше.

## способ улучшить алгоритм№1


#### Если слово используется 1 раз в негативных отзывах, но, допустим, 60 в положительных, то, кажется, вполне обоснованно можно посчитать его "положительным". Мне кажется достаточным, чтобы слово встречалось хотя бы в 4 раза чаще в одной из тональностей (на разных тестовых разные параметры дают разные результаты, в принципе может быть полезно и еще меньшее отношение), но этот параметр можно менять и смотреть как меняется результат (на 10 тестовых примерах сейчас такое особенно не оценить). Немного изменим функцию раскидывающую слова по положительным и негативным множествам. Особенно здорово это должно нивелировать нюанс что я описал в комментарии выше

In [310]:
for key in dict_pos.keys():
    if dict_pos[key] > 1:
        if key in dict_neg.keys():
            if dict_pos[key] > 4 * dict_neg[key]:
                final_set_pos.add(key)
        else:
            final_set_pos.add(key)

for key in dict_neg.keys():
    if dict_neg[key] > 1:
        if key in dict_pos.keys():
            if dict_neg[key] > 4 * dict_pos[key]:
                final_set_neg.add(key)
        else:
            final_set_neg.add(key)
            
#в среднем так немного увеличивается точность +- на 0,1

## способ улучшить алгоритм№2

#### Если считать точность так, что каждый верный ответ с точки зрения бинарной классификации прибавляет балл, каждый неправильный отнимает балл, а нейтральный ответ (определение тональности как нейтральной) не снимает и не добавляет, тогда ввод нейтральной тональности становится очень важным, ведь если у модели мало уверенности, то лучше воздержаться от предсказания, а не предсказывать с высоким шансом ошибки. Кажется так считать точность супер обоснованно, ведь получать противположное определение тональности нам совсем не хочется ни в каких задачах. Тогда для каждого комментария можно вести учет "положительных" и "негативных" слов, и, если разница между ними не слишком большая, тогда определять тональность нейтрально. Перепишем эту часть кода

In [325]:
for text in X_test:
    # Создаем счетчики для положительных и негативынх слов
    pos_counter = 0
    neg_counter = 0
    for token in text:
        if token in final_set_pos:
            pos_counter += 1
        elif token in final_set_neg:
            neg_counter += 1
    #опять же параметры можно менять, добиавясь оптимальных
    if pos_counter > 1 and pos_counter > 1.5 * neg_counter:
        y_pred.append('pos')
    elif neg_counter > 1 and neg_counter > 1.5 * pos_counter:
        y_pred.append('neg')
    else:
        y_pred.append('neut')

In [326]:
accuracy = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test[i]:
        #т.к. примеров всего 10
        accuracy += 0.1
    elif (y_pred[i] == 'pos' and y_test[i] == 'neg'):
        accuracy -= 0.1
    elif (y_pred[i] == 'neg' and y_test[i] == 'pos'):
        accuracy -= 0.1
        
#это более жесткий способ считать точность, тем не менее результаты +- такие же